Our objective is to deliver percentage completion rates for tasks to show progress towards our implementation goals.
First, we need to create a dataframe from an excel file which we obtain by exporting our implementation tasks from Wrike.
After importing our datafile we select only the Folder, Parent Task and Status columns.

In [119]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
file = 'Tenneco.xls'
df = pd.read_excel(file)
newdf = df.ix[:,['Folder','Parent task','Status']]
newdf.head()

,Folder,Parent task,Status
0,/,NaN,NaN
1,/Angola/,NaN,Completed
2,/Angola/,Initial Training,Completed
3,/Angola/,Initial Training,Completed
4,/Angola/,Initial Training,Completed


We clean the Folder column and strip out the excess "/" characters bookending the string. We leave any within the string to designate between Implementation and Trial tasks.

In [120]:
newdf['Folder'] = newdf['Folder'].map(lambda x: x.lstrip('/').rstrip('/'))
newdf = newdf.dropna()
newdf.head()

,Folder,Parent task,Status
2,Angola,Initial Training,Completed
3,Angola,Initial Training,Completed
4,Angola,Initial Training,Completed
5,Angola,Initial Training,Completed
6,Angola,Initial Training,Completed


We get a total number of tasks by counting the number of items grouped by Folder. We also get totals for each status by aggregating the tasks grouped by folder. We then create a new table and join our totals to the aggregate data.

In [122]:
aggregations = {
    'Status':{
        'Status Count': 'count',
    },
}
def addTotals(row):
    if row['Status', ''] == 'Completed':
        tempdf = totals.loc[totals['Folder'] == row[0]]
        tempdf = tempdf.ix[:,1]
        return tempdf.iloc[0]
    else:
        return ''
#toCalc = newdf.apply(newtest,axis=1)
totals = newdf.groupby(['Folder']).count()
totals.reset_index(inplace=True)
grouped = newdf.groupby(['Folder', 'Status']).agg(aggregations)
grouped.reset_index(inplace=True)
#grouped = grouped.drop(grouped.index[0],axis=0)
#grouped[totals.columns] = totals
#new = grouped.join(totals)
#new
grouped['Total Tasks'] = grouped.apply(addTotals,axis=1)

The join operation adds in an excess column so we drop it to clean the table.

In [113]:
#new = new.drop(new.columns[2],axis=1)
#sns.swarmplot(x='(Status, Status Count)', y= 'Parent task', data=new)
grouped

Folder     Status              Total Tasks
                            Status Count            
0         Angola     Active            3            
1         Angola  Cancelled            1            
2         Angola  Completed           14          18
3         Celaya     Active            2            
4         Celaya  Cancelled            1            
5         Celaya  Completed           16          19
6   Harrisonburg     Active            1            
7   Harrisonburg  Cancelled            1            
8   Harrisonburg  Completed           18          20
9       Hartwell  Cancelled            1            
10      Hartwell  Completed           16          17
11     Kettering  Cancelled            1            
12     Kettering  Completed           16          17
13      Napoleon     Active           10            
14      Napoleon  Completed           14          24
15    Owen Sound     Active            6            
16    Owen Sound  Completed           17          23
17     Paragould  Cancelled            1            
18     Paragould  Completed           20          21
19       Reynosa     Active            5            
20       Reynosa  Cancelled            1            
21       Reynosa  Completed           15          21

In [114]:
list(grouped.columns.values)

[('Folder', ''),
 ('Status', ''),
 ('Status', 'Status Count'),
 ('Total Tasks', '')]

We rename the aggregate totals for status to clean up the header.

In [124]:
grouped.rename(columns={('Folder', ''):('Site', '')}, inplace=True)
#grouped.columns.values[0] = ('Site', '')
grouped

Folder     Status              Total Tasks
                            Status Count            
0         Angola     Active            3            
1         Angola  Cancelled            1            
2         Angola  Completed           14          18
3         Celaya     Active            2            
4         Celaya  Cancelled            1            
5         Celaya  Completed           16          19
6   Harrisonburg     Active            1            
7   Harrisonburg  Cancelled            1            
8   Harrisonburg  Completed           18          20
9       Hartwell  Cancelled            1            
10      Hartwell  Completed           16          17
11     Kettering  Cancelled            1            
12     Kettering  Completed           16          17
13      Napoleon     Active           10            
14      Napoleon  Completed           14          24
15    Owen Sound     Active            6            
16    Owen Sound  Completed           17          23
17     Paragould  Cancelled            1            
18     Paragould  Completed           20          21
19       Reynosa     Active            5            
20       Reynosa  Cancelled            1            
21       Reynosa  Completed           15          21

We create a new column and calculate the percenatage for each status.

In [125]:
def percent(row):
    if row[('Total Tasks', '')] != '':
        return (row['Status','Status Count']/row['Total Tasks','']*100).round(2).astype(str) + '%'
    else:
        return ''
#        #return (new[('Status Count')]/new['Parent task']*100).round(2).astype(str) + '%'
grouped['Completion %'] = grouped.apply(percent,axis=1)

#new['Percent of Total'] = new.apply(percent, axis=1)
#new.index['Completed']((new[('Status Count')]/new['Parent task'])*100).round(2).astype(str) + '%'
#new = new.drop(new.index[-1], axis=0)
#new
completedSum = 0

#addrow = pd.DataFrame(np.array['Totals','',])

totalTaskCount = grouped['Status', 'Status Count'].sum()
completedTaskCount = 0
for x in range(grouped.shape[0]):
    if grouped.get_value(x,('Status','')) == 'Completed':
        completedTaskCount += grouped.get_value(x,('Status','Status Count'))
totalPercentComplete = ((completedTaskCount/totalTaskCount)*100).round(2).astype(str)+'%'

In [129]:
grouped.loc[grouped.shape[0]]=['Completed / Total','',completedTaskCount,totalTaskCount,totalPercentComplete]
grouped

Folder     Status              Total Tasks Completion %
                                   Status Count                         
0                Angola     Active            3                         
1                Angola  Cancelled            1                         
2                Angola  Completed           14          18       77.78%
3                Celaya     Active            2                         
4                Celaya  Cancelled            1                         
5                Celaya  Completed           16          19       84.21%
6          Harrisonburg     Active            1                         
7          Harrisonburg  Cancelled            1                         
8          Harrisonburg  Completed           18          20        90.0%
9              Hartwell  Cancelled            1                         
10             Hartwell  Completed           16          17       94.12%
11            Kettering  Cancelled            1                         
12            Kettering  Completed           16          17       94.12%
13             Napoleon     Active           10                         
14             Napoleon  Completed           14          24       58.33%
15           Owen Sound     Active            6                         
16           Owen Sound  Completed           17          23       73.91%
17            Paragould  Cancelled            1                         
18            Paragould  Completed           20          21       95.24%
19              Reynosa     Active            5                         
20              Reynosa  Cancelled            1                         
21              Reynosa  Completed           15          21       71.43%
22    Completed:/Total:                     146         180       81.11%
23  Completed: / Total:                     146         180       81.11%
24    Completed / Total                     146         180       81.11%
25    Completed | Total                     146         180       81.11%

In [118]:
grouped.to_csv('out.csv')

In [88]:
grouped.dtypes

Folder                        object
Status                        object
              Status Count     int64
Total Tasks                   object
Completion %                  object
dtype: object